# Chapter 24: Other Blockchain Platforms

---

Ethereum may be the most popular smart contract platform, but it's far from the only one. A diverse ecosystem of blockchains has emerged, each with unique architectures, trade-offs, and programming models. Some, like Solana, prioritize high throughput; others, like Cardano, emphasize formal verification; while platforms like Polkadot and Cosmos focus on interoperability. Enterprise solutions like Hyperledger Fabric offer permissioned networks for business use. Understanding these alternatives is essential for choosing the right tool for your project and for appreciating the broader blockchain landscape.

In this chapter, we'll explore the most prominent blockchain platforms beyond Ethereum, diving into their core concepts, development environments, and unique features. By the end, you'll have a solid foundation to evaluate and build on these platforms.

---

## 24.1 Bitcoin Deep Dive

Bitcoin is the original blockchain, introduced in 2009 by Satoshi Nakamoto. While not designed for complex smart contracts, its design—particularly the UTXO model and Bitcoin Script—has influenced many subsequent blockchains. Recent innovations like the Lightning Network and Ordinals have expanded Bitcoin's capabilities.

### 24.1.1 UTXO Model Explained

Bitcoin uses the **Unspent Transaction Output (UTXO)** model, which differs fundamentally from Ethereum's account-based model.

**How UTXO works:**
- A transaction consumes existing UTXOs as inputs and creates new UTXOs as outputs.
- Each UTXO is an indivisible chunk of bitcoin associated with a locking script (typically requiring a signature).
- Your balance is the sum of all UTXOs controlled by your keys.

```
UTXO Transaction Example:

Before Transaction:
Alice has UTXO of 10 BTC (requires Alice's signature)
Bob has UTXO of 5 BTC (requires Bob's signature)

Transaction: Alice sends 3 BTC to Bob.
- Input: Alice's UTXO (10 BTC)
- Outputs:
  - 3 BTC to Bob (locked with Bob's key)
  - 7 BTC change to Alice (locked with Alice's key)

After Transaction:
Alice's 10 BTC UTXO is spent (destroyed)
New UTXOs: 3 BTC (Bob), 7 BTC (Alice)
Bob's original 5 BTC UTXO remains unspent.
```

**Advantages of UTXO:**
- **Parallelism**: Multiple UTXOs can be spent independently, enabling concurrent transactions.
- **Privacy**: It's harder to link addresses (though heuristics exist).
- **Scalability**: Stateless validation (no global state).
- **Simpler light clients**: Only need to track UTXOs relevant to you.

**Disadvantages:**
- **Complexity**: Managing UTXOs requires coin selection logic.
- **Smart contracts**: Harder to implement stateful contracts (though possible with covenants and taproot).

### 24.1.2 Bitcoin Script

Bitcoin's smart contract language is **Script**, a stack-based, non-Turing-complete language. It's intentionally limited to prevent infinite loops and ensure deterministic execution.

**Key properties:**
- No loops (no `jump` opcodes except conditional).
- Limited opcodes (~200, but many disabled).
- Stateless (no persistent storage).
- Executed by every node during transaction validation.

**Typical script: Pay-to-PubKey-Hash (P2PKH)**
```
Locking script (scriptPubKey):
OP_DUP OP_HASH160 <pubKeyHash> OP_EQUALVERIFY OP_CHECKSIG

Unlocking script (scriptSig):
<signature> <pubKey>
```

**Execution:**
1. Stack: [signature, pubKey]
2. OP_DUP duplicates pubKey.
3. OP_HASH160 hashes the pubKey.
4. Compare with pubKeyHash; if equal, continue.
5. OP_CHECKSIG verifies signature against pubKey and transaction.

**Modern extensions:**
- **SegWit**: Fixes transaction malleability and increases block capacity.
- **Taproot**: Enables more complex scripts with Schnorr signatures and MAST, improving privacy and efficiency.
- **Ordinals**: Allow inscribing data on satoshis, enabling NFTs on Bitcoin.

### 24.1.3 Lightning Network

The Lightning Network is a layer-2 solution for Bitcoin enabling instant, low-cost micropayments. It uses payment channels (see Chapter 16) to move transactions off-chain.

**How it works:**
1. Two parties open a channel by funding a multisig address on-chain.
2. They exchange signed commitment transactions updating balances off-chain.
3. The channel can be closed by broadcasting the latest commitment transaction.
4. Payments can be routed through multiple channels using HTLCs (Hashed TimeLock Contracts).

**Lightning in practice:**
- Wallets: Phoenix, Breez, Zap.
- Nodes: LND (Lightning Network Daemon), c-lightning, Eclair.
- Routing nodes earn fees for forwarding payments.

### 24.1.4 Ordinals and BRC-20

**Ordinals** assign a serial number to each satoshi (1 BTC = 100,000,000 satoshis) based on the order it was mined. Users can "inscribe" data (images, text, etc.) onto individual satoshis, creating Bitcoin-native NFTs.

**BRC-20** is an experimental token standard using Ordinals and JSON inscriptions to create fungible tokens. It's similar to Ethereum's ERC-20 but on Bitcoin. However, it's limited by Bitcoin's script capabilities and relies on off-chain indexing.

**Example BRC-20 inscription (JSON):**
```json
{
  "p": "brc-20",
  "op": "deploy",
  "tick": "ordi",
  "max": "21000000",
  "lim": "1000"
}
```

These innovations show Bitcoin's evolution beyond a simple store of value.

---

## 24.2 Solana

Solana is a high-performance blockchain designed for mass adoption. It achieves thousands of transactions per second through a unique combination of technologies: Proof of History (PoH), Tower BFT, and parallel transaction processing.

### 24.2.1 Architecture Overview

Solana's architecture is built around **Proof of History (PoH)**—a cryptographic clock that timestamps events before consensus. PoH creates a historical record proving that an event occurred at a specific moment, reducing communication overhead.

**Key components:**
- **PoH Generator**: Produces a sequence of hashes where each hash is the output of the previous hash, creating a verifiable time source.
- **Tower BFT**: A PoS consensus mechanism leveraging PoH to reduce message passing.
- **Gulf Stream**: Mempool-less transaction forwarding; transactions are sent directly to validators.
- **Sealevel**: Parallel smart contract runtime (multiple contracts can execute concurrently).
- **Cloudbreak**: Horizontally scaled accounts database.

```
Solana Transaction Flow:
User → RPC → Leader (Validator) → Gulf Stream → Sealevel (parallel exec) → PoH timestamp → Consensus → Ledger
```

**Accounts model:**
- Each account has a balance and can store data (up to 10 MB).
- Programs (smart contracts) are stateless; they operate on accounts passed as references.
- Rent: Accounts must maintain a minimum balance or be garbage-collected.

### 24.2.2 Rust Programming for Solana

Solana smart contracts (called **programs**) are written in Rust or C. Rust is the primary language due to its safety and performance.

**A simple Solana program (Rust):**
```rust
use solana_program::{
    account_info::AccountInfo,
    entrypoint,
    entrypoint::ProgramResult,
    pubkey::Pubkey,
    msg,
};

// Declare the program's entrypoint
entrypoint!(process_instruction);

// Program logic
fn process_instruction(
    program_id: &Pubkey,
    accounts: &[AccountInfo],
    instruction_data: &[u8],
) -> ProgramResult {
    msg!("Hello, Solana!");
    Ok(())
}
```

**Key concepts:**
- **Accounts**: All data is stored in accounts. Programs are accounts too.
- **Cross-Program Invocations (CPI)**: One program can call another.
- **PDAs (Program Derived Addresses)**: Accounts owned by programs without a corresponding private key, used for deterministic addresses.

### 24.2.3 Anchor Framework

Anchor is a framework for Solana development that simplifies writing, testing, and deploying programs. It provides Rust macros, IDL generation, and client libraries.

**Example Anchor program:**
```rust
use anchor_lang::prelude::*;

declare_id!("Fg6PaFpoGXkYsidMpWTK6W2BeZ7FEfcYkg476zPFsLnS");

#[program]
pub mod my_program {
    use super::*;

    pub fn initialize(ctx: Context<Initialize>, data: u64) -> Result<()> {
        let my_account = &mut ctx.accounts.my_account;
        my_account.data = data;
        Ok(())
    }
}

#[derive(Accounts)]
pub struct Initialize<'info> {
    #[account(init, payer = user, space = 8 + 8)]
    pub my_account: Account<'info, MyAccount>,
    #[account(mut)]
    pub user: Signer<'info>,
    pub system_program: Program<'info, System>,
}

#[account]
pub struct MyAccount {
    pub data: u64,
}
```

Anchor handles account validation, serialization, and security checks.

### 24.2.4 Developing on Solana

**Setup:**
```bash
# Install Solana CLI
sh -c "$(curl -sSfL https://release.solana.com/v1.16.0/install)"
# Install Anchor
cargo install --git https://github.com/coral-xyz/anchor anchor-cli
```

**Build and deploy:**
```bash
anchor build
anchor deploy
```

**Testing:** Anchor generates TypeScript tests using Mocha.

```typescript
import * as anchor from "@project-serum/anchor";
import { Program } from "@project-serum/anchor";
import { MyProgram } from "../target/types/my_program";

describe("my_program", () => {
  anchor.setProvider(anchor.AnchorProvider.env());
  const program = anchor.workspace.MyProgram as Program<MyProgram>;

  it("Initializes with data", async () => {
    const data = new anchor.BN(42);
    const tx = await program.methods.initialize(data).rpc();
    console.log("Transaction signature", tx);
  });
});
```

Solana's mainnet handles high throughput, but development is more complex than Ethereum due to the account model and parallel execution considerations.

---

## 24.3 Cardano

Cardano is a proof-of-stake blockchain platform founded on peer-reviewed research and formal methods. It emphasizes security, sustainability, and interoperability.

### 24.3.1 Extended UTXO Model

Cardano uses an **Extended UTXO (EUTXO)** model, which enhances Bitcoin's UTXO with the ability to carry additional data and execute scripts.

**Key features:**
- UTXOs can contain arbitrary data (datums).
- Scripts (validators) decide if a UTXO can be spent based on the transaction and the datum.
- Transactions can have multiple inputs and outputs, enabling complex stateful logic.

**EUTXO advantages:**
- **Predictability**: Scripts are pure; given the same inputs, the same output.
- **Parallelism**: Multiple UTXOs can be processed simultaneously.
- **Off-chain computation**: Much of the logic can be handled off-chain, with on-chain validation only.

**Comparison with account model:**
| Account Model (Ethereum) | EUTXO (Cardano) |
|--------------------------|-----------------|
| Global state | Local state per UTXO |
| Reentrancy risks | Deterministic, no reentrancy |
| Sequential execution | Parallelizable |
| Higher gas costs for complex logic | Lower costs for similar logic |

### 24.3.2 Plutus Smart Contracts

Plutus is Cardano's smart contract platform. Contracts are written in Haskell (or Plutus Tx, a subset of Haskell) and compiled to Plutus Core, which runs on-chain.

**Plutus contract structure:**
- **On-chain code**: Validator script (Haskell) that validates transactions.
- **Off-chain code**: Client-side code that constructs and submits transactions.

**Example: Simple vesting contract**
```haskell
{-# INLINABLE mkValidator #-}
mkValidator :: BuiltinData -> BuiltinData -> BuiltinData -> ()
mkValidator _ _ _ = ()

validator :: Validator
validator = mkValidatorScript $$(compile [|| mkValidator ||])
```

The actual logic would check beneficiary, deadline, etc.

**Development tools:**
- **Plutus Playground**: Web-based IDE for testing.
- **Plutus Application Backend (PAB)**: For running off-chain code.
- **Cardano CLI**: For interacting with the node.

### 24.3.3 Haskell Development

Cardano's core is written in Haskell, and smart contracts require Haskell knowledge. This steep learning curve is a barrier but also ensures strong type safety and correctness.

**Example: Defining a parameterized validator**
```haskell
{-# INLINABLE mkVestingValidator #-}
mkVestingValidator :: PubKeyHash -> POSIXTime -> BuiltinData -> BuiltinData -> BuiltinData -> ()
mkVestingValidator beneficiary deadline _ _ _ = traceIfFalse "not beneficiary" (txSignedBy info beneficiary) `and` traceIfFalse "deadline not reached" (to deadline `contains` from ??)
```

Cardano's development ecosystem is maturing with tools like **Plutus-apps**, **Marlowe** (financial contracts DSL), and **Aiken** (a Rust-like language for Cardano).

---

## 24.4 Polkadot and Substrate

Polkadot is a heterogeneous multi-chain protocol enabling different blockchains (parachains) to interoperate under a common security umbrella. **Substrate** is the framework for building these blockchains.

### 24.4.1 Parachain Architecture

Polkadot's architecture consists of:
- **Relay Chain**: The central chain providing security, consensus, and cross-chain communication.
- **Parachains**: Sovereign blockchains that connect to the Relay Chain, benefiting from its security.
- **Parathreads**: Pay-as-you-go parachains.
- **Bridges**: Connect external chains (Ethereum, Bitcoin) to Polkadot.

```
┌─────────────────────────────────────────────────────────────┐
│                     POLKADOT RELAY CHAIN                     │
│  • Consensus (NPoS)                                         │
│  • Finality (GRANDPA)                                       │
│  • Cross-chain messaging (XCMP)                             │
└────────────┬─────────────────────────────┬──────────────────┘
             │                             │
    ┌────────▼────────┐           ┌────────▼────────┐
    │  Parachain A    │           │  Parachain B    │
    │ (e.g., Acala)   │           │ (e.g., Moonbeam)│
    └─────────────────┘           └─────────────────┘
```

**Slot auctions:** Parachains bid for a slot on the Relay Chain for a fixed period (e.g., 6 months), bonding DOT tokens.

### 24.4.2 Substrate Framework

Substrate is a modular framework for building blockchains. It provides:
- **Runtime modules (pallets)**: Pre-built components (balances, staking, governance).
- **FRAME**: The system for composing pallets.
- **Client**: Networking, consensus, database layers.

**Example: A simple Substrate pallet**
```rust
#![cfg_attr(not(feature = "std"), no_std)]

use frame_support::{pallet, dispatch::DispatchResult};
use frame_system::pallet_prelude::*;

#[pallet]
pub mod pallet {
    use super::*;
    use frame_support::pallet_prelude::*;

    #[pallet::config]
    pub trait Config: frame_system::Config {}

    #[pallet::pallet]
    pub struct Pallet<T>(_);

    #[pallet::storage]
    pub type Something<T> = StorageValue<_, u32>;

    #[pallet::call]
    impl<T: Config> Pallet<T> {
        #[pallet::weight(10_000)]
        pub fn set_something(origin: OriginFor<T>, value: u32) -> DispatchResult {
            let _ = ensure_signed(origin)?;
            Something::<T>::put(value);
            Ok(())
        }
    }
}
```

**Build and run:**
```bash
cargo build --release
./target/release/node-template --dev
```

Substrate chains can be standalone or connect to Polkadot as parachains.

### 24.4.3 Cross-Chain Communication

**XCMP (Cross-Chain Message Passing)** allows parachains to exchange messages. Messages are sent via the Relay Chain but not executed there; they are passed to the destination parachain.

**HRMP (Horizontal Relay-routed Message Passing)** is a temporary protocol until XCMP is fully implemented.

**XCM (Cross-Consensus Message Format)**: The language for expressing what should be done with a message (transfer tokens, call functions, etc.). XCM is not specific to Polkadot; it's a format for cross-consensus communication.

**Example XCM instruction:**
```
WithdrawAsset(Here, 100)
BuyExecution { fees: 100, weight: 100000 }
DepositAsset { assets: Wild, beneficiary: AccountId32(id) }
```

Polkadot's ecosystem includes smart contract parachains like **Moonbeam** (EVM-compatible) and **Astar**, allowing Solidity developers to deploy to Polkadot.

---

## 24.5 Cosmos

Cosmos is an ecosystem of interoperable blockchains, often called the "Internet of Blockchains." Its core innovation is the **IBC (Inter-Blockchain Communication)** protocol, enabling chains to transfer assets and data.

### 24.5.1 Tendermint Consensus

**Tendermint** is a Byzantine Fault Tolerant (BFT) consensus engine that powers many Cosmos chains. It provides:
- **Consensus**: Practical Byzantine Fault Tolerance (PBFT) with 1/3 fault tolerance.
- **ABCI (Application Blockchain Interface)**: Separates consensus from application logic.

```
Tendermint Architecture:
┌─────────────────┐     ┌─────────────────┐
│   Application   │     │   Application   │
│   (Cosmos SDK)  │◄───▶│   (Cosmos SDK)  │
└─────────────────┘     └─────────────────┘
        ▲                       ▲
        │                       │
        ▼                       ▼
┌─────────────────┐     ┌─────────────────┐
│   Tendermint    │     │   Tendermint    │
│    (Consensus   │     │    (Consensus   │
│      + P2P)     │     │      + P2P)     │
└─────────────────┘     └─────────────────┘
```

**Features:**
- Fast finality (few seconds).
- High performance (thousands of TPS).
- Easy to build applications with Cosmos SDK.

### 24.5.2 IBC (Inter-Blockchain Communication)

IBC is a protocol for reliable, authenticated communication between heterogeneous blockchains. It enables token transfers, cross-chain calls, and more.

**How IBC works:**
1. **Light clients**: Each chain maintains a light client of the other chain (using Tendermint's consensus proofs).
2. **Relayers**: Off-chain processes relay packets between chains.
3. **Handshakes**: Establish a connection and channel.
4. **Packet flow**: Data is sent with timeouts and acknowledgments.

```
Chain A                Relayer                Chain B
  |                        |                      |
  |───sendPacket──────────▶|                      |
  |                        |───recvPacket────────▶|
  |                        |                      |
  |                        |───writeAcknowledgment│
  |◀───ackPacket───────────|                      |
```

IBC has enabled a vibrant ecosystem of interconnected chains: Cosmos Hub, Osmosis (DEX), Juno (smart contracts), and many others.

### 24.5.3 Cosmos SDK

The **Cosmos SDK** is a framework for building application-specific blockchains. It provides:
- **Modules**: Pre-built components (auth, bank, staking, gov).
- **BaseApp**: The ABCI application.
- **Keepers**: Modules that manage state.
- **Protobuf**: For encoding and gRPC.

**Example: Creating a simple module**
```go
package mymodule

import (
    sdk "github.com/cosmos/cosmos-sdk/types"
)

type Keeper struct {
    storeKey sdk.StoreKey
}

func (k Keeper) SetData(ctx sdk.Context, key string, value string) {
    store := ctx.KVStore(k.storeKey)
    store.Set([]byte(key), []byte(value))
}

func (k Keeper) GetData(ctx sdk.Context, key string) string {
    store := ctx.KVStore(k.storeKey)
    return string(store.Get([]byte(key)))
}
```

**Cosmos ecosystem:**
- **Cosmos Hub**: The first hub, uses ATOM token.
- **Osmosis**: Leading DEX with IBC-enabled pools.
- **Secret Network**: Privacy-preserving smart contracts.
- **Celestia**: Modular data availability layer.

Developers can build sovereign chains tailored to their application, avoiding the constraints of a shared execution environment.

---

## 24.6 BNB Smart Chain

BNB Smart Chain (BSC) is a blockchain parallel to Binance Chain, designed for smart contracts and EVM compatibility. It's popular for its low fees and fast block times, though at the cost of higher centralization.

### 24.6.1 EVM Compatibility

BSC is fully EVM-compatible, meaning Ethereum smart contracts can be deployed with minimal changes. It supports the same tooling (Hardhat, Truffle, MetaMask) and programming languages (Solidity).

**Key differences from Ethereum:**
- **Block time**: ~3 seconds (vs. 12-15 seconds).
- **Gas fees**: Much lower (often fractions of a cent).
- **Consensus**: Proof of Staked Authority (PoSA).

**Network configuration for MetaMask:**
```
Network Name: BNB Smart Chain
RPC URL: https://bsc-dataseed.binance.org/
Chain ID: 56
Symbol: BNB
Block Explorer: https://bscscan.com
```

### 24.6.2 Proof of Staked Authority

BSC uses **Proof of Staked Authority (PoSA)**, a hybrid of PoA and PoS. A set of validators (21) are elected by staking BNB. They produce blocks in a round-robin fashion.

**Validator requirements:**
- Run a full node.
- Stake a significant amount of BNB.
- Maintain high uptime.

**Pros:**
- Fast finality.
- Low energy consumption.
- High throughput (~100 TPS).

**Cons:**
- Centralization (only 21 validators).
- Censorship potential if validators collude.

### 24.6.3 Developing on BSC

Development on BSC is nearly identical to Ethereum. You can use Hardhat with the BSC network.

**Hardhat config for BSC:**
```javascript
module.exports = {
  solidity: "0.8.19",
  networks: {
    bscTestnet: {
      url: "https://data-seed-prebsc-1-s1.binance.org:8545",
      accounts: [process.env.PRIVATE_KEY],
    },
  },
};
```

**Deploy:**
```bash
npx hardhat run scripts/deploy.js --network bscTestnet
```

BSC also has its own ecosystem with bridges to Ethereum, making it a common choice for projects seeking lower fees and faster transactions.

---

## 24.7 Avalanche

Avalanche is a platform for launching decentralized applications and custom blockchains with high throughput and fast finality. It introduces a novel consensus protocol and a multi-chain architecture.

### 24.7.1 Multi-Chain Architecture

Avalanche consists of three built-in chains, with the ability to create more via Subnets.

- **X-Chain (Exchange Chain)**: Creates and trades assets (using Avalanche consensus).
- **C-Chain (Contract Chain)**: EVM-compatible chain for smart contracts.
- **P-Chain (Platform Chain)**: Coordinates validators, Subnets, and staking.

```
┌─────────────────────────────────────────────────────────────┐
│                         PRIMARY NETWORK                      │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐         │
│  │   X-Chain   │  │   C-Chain   │  │   P-Chain   │         │
│  │ (DAG, asset │  │ (EVM, smart │  │ (validators,│         │
│  │   creation) │  │  contracts) │  │  subnets)   │         │
│  └─────────────┘  └─────────────┘  └─────────────┘         │
└─────────────────────────────────────────────────────────────┘
                              │
                    ┌─────────┴─────────┐
                    │                   │
                    ▼                   ▼
            ┌─────────────┐     ┌─────────────┐
            │  Subnet A   │     │  Subnet B   │
            │ (custom VM) │     │ (custom VM) │
            └─────────────┘     └─────────────┘
```

**Avalanche Consensus**: A novel protocol based on repeated random subsampling, achieving high throughput and low latency without sacrificing decentralization.

### 24.7.2 Subnets

A **Subnet** is a dynamic set of validators working together to achieve consensus on one or more blockchains. Each blockchain is validated by exactly one Subnet, but a Subnet can validate many blockchains.

**Benefits:**
- Customizable: Create a blockchain with its own VM, gas token, and rules.
- Scalability: Subnets can run in parallel without interfering.
- Compliance: Can restrict participants for regulatory needs.

### 24.7.3 Developing on Avalanche

**C-Chain (EVM)**: Deploy Solidity contracts using familiar tools.

```javascript
// Hardhat config
module.exports = {
  solidity: "0.8.19",
  networks: {
    avalancheFuji: {
      url: "https://api.avax-test.network/ext/bc/C/rpc",
      accounts: [process.env.PRIVATE_KEY],
    },
  },
};
```

**Custom VM development**: Avalanche supports building Subnets with custom VMs using the **Avalanche Virtual Machine (AVM)**. Developers can use **Hypersdk** or **Subnet-EVM** (pre-built EVM) to launch custom chains quickly.

**Avalanche Ecosystem:**
- **Trader Joe**: DEX.
- **Benqi**: Lending.
- **Aave**: Deployed on C-Chain.

Avalanche's multi-chain approach and subnet flexibility make it attractive for projects needing sovereignty.

---

## 24.8 Hyperledger (Enterprise Blockchain)

Hyperledger is an umbrella project of open-source blockchain frameworks and tools hosted by the Linux Foundation, focused on enterprise use cases. Unlike public blockchains, Hyperledger frameworks are permissioned, prioritizing privacy, scalability, and identity.

### 24.8.1 Hyperledger Fabric

Hyperledger Fabric is the most widely used Hyperledger framework. It's designed for modular architecture, pluggable consensus, and support for confidential transactions.

**Key concepts:**
- **Peer nodes**: Maintain ledger and chaincode.
- **Orderer nodes**: Order transactions and create blocks.
- **Channel**: Private sub-network between specific members.
- **Chaincode**: Smart contracts (in Go, Java, JavaScript).
- **World state**: Current state of assets (e.g., LevelDB, CouchDB).

```
Fabric Architecture:
┌─────────────────────────────────────────────────────────────┐
│                         Organization A                       │
│  ┌─────────────┐     ┌─────────────┐                        │
│  │   Peer 1    │     │   Peer 2    │                        │
│  │ (chaincode) │     │ (chaincode) │                        │
│  └─────────────┘     └─────────────┘                        │
└────────────────────────────┬────────────────────────────────┘
                             │
                     ┌───────┴───────┐
                     │   Orderer     │
                     │  (consensus)  │
                     └───────────────┘
┌────────────────────────────┬────────────────────────────────┐
│                         Organization B                       │
│  ┌─────────────┐     ┌─────────────┐                        │
│  │   Peer 1    │     │   Peer 2    │                        │
│  │ (chaincode) │     │ (chaincode) │                        │
│  └─────────────┘     └─────────────┘                        │
└─────────────────────────────────────────────────────────────┘
```

**Consensus**: Pluggable; common implementations:
- **Raft**: Crash fault-tolerant (CFT), simple.
- **Kafka** (deprecated).
- **Solo** (for development).
- **BFT** (planned for future).

### 24.8.2 Permissioned Networks

Fabric networks are permissioned—all participants have identities managed by a **Membership Service Provider (MSP)** (often using X.509 certificates). This enables:
- **Privacy**: Transactions can be private to a channel.
- **Auditability**: All actions are linked to identities.
- **Compliance**: Meet regulatory requirements.

**Transaction flow:**
1. Client proposes transaction to endorsing peers.
2. Peers simulate chaincode and return endorsement signatures.
3. Client sends endorsed transaction to orderer.
4. Orderer creates block and distributes to peers.
5. Peers validate and commit to ledger.

### 24.8.3 Chaincode Development

Chaincode can be written in Go, Java, or JavaScript.

**Example chaincode in Go (simple asset transfer):**
```go
package main

import (
    "fmt"
    "github.com/hyperledger/fabric-chaincode-go/shim"
    "github.com/hyperledger/fabric-protos-go/peer"
)

type SimpleAsset struct {}

func (t *SimpleAsset) Init(stub shim.ChaincodeStubInterface) peer.Response {
    args := stub.GetStringArgs()
    if len(args) != 2 {
        return shim.Error("Incorrect arguments. Expecting key and value")
    }
    err := stub.PutState(args[0], []byte(args[1]))
    if err != nil {
        return shim.Error(fmt.Sprintf("Failed to create asset: %s", args[0]))
    }
    return shim.Success(nil)
}

func (t *SimpleAsset) Invoke(stub shim.ChaincodeStubInterface) peer.Response {
    fn, args := stub.GetFunctionAndParameters()
    if fn == "set" {
        return t.set(stub, args)
    } else if fn == "get" {
        return t.get(stub, args)
    }
    return shim.Error("Invalid function")
}

func (t *SimpleAsset) set(stub shim.ChaincodeStubInterface, args []string) peer.Response {
    if len(args) != 2 {
        return shim.Error("Incorrect arguments. Expecting key and value")
    }
    err := stub.PutState(args[0], []byte(args[1]))
    if err != nil {
        return shim.Error(err.Error())
    }
    return shim.Success(nil)
}

func (t *SimpleAsset) get(stub shim.ChaincodeStubInterface, args []string) peer.Response {
    if len(args) != 1 {
        return shim.Error("Incorrect arguments. Expecting key")
    }
    value, err := stub.GetState(args[0])
    if err != nil {
        return shim.Error(err.Error())
    }
    if value == nil {
        return shim.Error("Asset not found")
    }
    return shim.Success(value)
}

func main() {
    err := shim.Start(new(SimpleAsset))
    if err != nil {
        fmt.Printf("Error starting chaincode: %s", err)
    }
}
```

**Deployment:**
- Package chaincode.
- Install on peers.
- Approve and commit on channel.

Fabric is widely used in finance, supply chain, and healthcare for private, permissioned networks.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 24 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Bitcoin: UTXO model, Bitcoin Script, Lightning Network,        │
│  Ordinals/BRC-20.                                              │
│                                                                 │
│  Solana: Proof of History, Rust-based programs, Anchor          │
│  framework, high throughput.                                    │
│                                                                 │
│  Cardano: Extended UTXO, Plutus (Haskell), formal methods.      │
│                                                                 │
│  Polkadot: Parachains, Substrate framework, XCMP cross-chain    │
│  communication.                                                 │
│                                                                 │
│  Cosmos: Tendermint consensus, IBC, Cosmos SDK for sovereign    │
│  chains.                                                        │
│                                                                 │
│  BNB Smart Chain: EVM-compatible, PoSA consensus, low fees.    │
│                                                                 │
│  Avalanche: Multi-chain (X/P/C), Subnets, EVM support.         │
│                                                                 │
│  Hyperledger Fabric: Permissioned, channels, chaincode in       │
│  Go/Java/JS, enterprise focus.                                 │
│                                                                 │
│  Each platform has unique trade-offs in decentralization,       │
│  performance, and developer experience. Choose based on         │
│  your application's needs.                                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 25 – Smart Contract Testing. We'll cover unit testing, integration testing, fuzzing, and tools like Hardhat, Foundry, and Echidna to ensure your contracts are secure and correct.